In [1]:
from random import randint, uniform, choice
from util import generate_problem, verify_witness

INF = 1e9

## Firstly we need a solver for the STP problem.

note here we are given a distance graph
(the generation is done in the genetic part, so we can reuse memory and save time when an interval selection is changed.)

In [2]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(E[i][j], E[i][k] + E[k][j])

    return E

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def solve_stp(graph):
    ''' graph is the constraint adjacency matrix '''
    
    d_graph = generate_d_graph(graph)
    
    return consistent(d_graph), get_min_sol(d_graph)

## The genetic part:

In [3]:
def random_gene(T):
    interval_selection = [randint(0, len(constr['intervals'])-1) for constr in T]
    return interval_selection

def update_graph(graph, gene):
    # set up graph using index 0 everywhere
    # so later we can change interval in O(1) time
    # (saves memory and time, and this same approach is especially beneficial in meta_walk)
    for i, constr in enumerate(T):
        interval_index = gene[i]
        
        i, j = constr['i'], constr['j']
        interval = constr['intervals'][interval_index]
        
        graph[i][j] = interval[1]
        graph[j][i] = -interval[0]
    
    return graph

def update_graph_at_constraint(graph, gene, constraint_index):
    ''' update_graph but updates only one constraint '''
    interval_index = gene[i]
    
    constr = T[constraint_index]
    
    i, j = constr['i'], constr['j']
    interval = constr['intervals'][interval_index]

    graph[i][j] = interval[1]
    graph[j][i] = -interval[0]
        
    return graph

def walk_gene(gene, T):
    # ignore all that have no choice
    # create list of plausible indices and filter it.
    candidate_indices = [j for j in range(len(T))]
    candidate_indices = list(filter(
        lambda x: len(T[x]['intervals']) > 1,
        candidate_indices
    ))
    
    idx = choice(candidate_indices)
    gene[idx] = randint(0, len(T[idx]['intervals'])-1)
    
    return gene

def meta_walk(T, max_iterations, max_flips):
    num_variables = max([max(t['i'], t['j']) for t in T])
    graph = discrete_graph(num_variables+1)
    
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T)
        graph = update_graph(graph, gene)
        
        for j in range(max_flips):
            # at each iteration we modify

            is_consistent, witness = solve_stp(graph)
            gene_failed = verify_witness(witness, T)

            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed

            if is_consistent:
                break
                
            gene = walk_gene(gene, T)
        
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [4]:
T = generate_problem()
meta_walk(T, 50, 50)

num iterations: 50
best gene: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 2, 0, 0, 0, 0, 1, 0, 0, 0]
constraints failed: 3 out of: 21
failed constraints: [{'i': 0, 'j': 9, 'intervals': [(-100, 100)]}, {'i': 2, 'j': 9, 'intervals': [(-73, -48), (79, 95)]}, {'i': 6, 'j': 9, 'intervals': [(21, 44)]}]


## Graphing effectiveness of walking a gene 

## WARNING: (TODO)
###  (it is hard to say if this proves anything -- it's basically the same as random)
### however, this can be analysed on a more larger-scale
### eg. walking is faster than regenerating the whole gene and graph

In [7]:
# problem sets:
SIMPLE = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 1,
    'scaling_factor': 1,
}

BINARY = {
    'constraint_probability': 0.25,
    'min_intervals': 2,
    'max_intervals': 2,
    'scaling_factor': 1,
}

FIVE_MAX = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 5,
    'scaling_factor': 1,
}

In [8]:
runs = 50
run_results = []

def test(T, max_flips):
    unsats=[]
    
    num_variables = max([max(t['i'], t['j']) for t in T])
    graph = discrete_graph(num_variables+1)
    
    gene = random_gene(T)
        
    for i in range(max_flips):
        graph = update_graph(graph, gene)
        
        is_consistent, witness = solve_stp(graph)
        gene_failed = verify_witness(witness, T)
        unsats += [len(gene_failed)]

        if is_consistent: break

        gene = walk_gene(gene, T)
        
    return unsats


for _ in range(runs):
    T = generate_problem(
        variables=20,
        **BINARY,
    )
    
    arr = test(T, max_flips=100)
    starting = arr[0]
    best = min(arr)
    if arr[0] > 0:
        decrease = (arr[0]-best) / arr[0]
    else:
        decrease = 1.0
    
    # TODO, problem: we do not know if it is even solvable!
    # will have to confirm with backtracking, or make sure to generate only solvable problems
    solved = best == 0
    
    print(starting, best, f'{decrease:.2f}', solved)

49 48 0.02 False
47 47 0.00 False
37 37 0.00 False
51 47 0.08 False
52 49 0.06 False
53 46 0.13 False
42 41 0.02 False
60 53 0.12 False
42 16 0.62 False
45 41 0.09 False
53 44 0.17 False
52 49 0.06 False
25 23 0.08 False
33 33 0.00 False
46 38 0.17 False
25 25 0.00 False
56 46 0.18 False
47 40 0.15 False
47 47 0.00 False
45 34 0.24 False
44 35 0.20 False
45 43 0.04 False
46 45 0.02 False
36 28 0.22 False
26 26 0.00 False
33 33 0.00 False
49 35 0.29 False
45 43 0.04 False
46 44 0.04 False
41 31 0.24 False
42 38 0.10 False
41 37 0.10 False
49 35 0.29 False
50 50 0.00 False
28 23 0.18 False
40 36 0.10 False
48 41 0.15 False
34 33 0.03 False
52 52 0.00 False
52 48 0.08 False
44 40 0.09 False
32 28 0.12 False
43 36 0.16 False
21 20 0.05 False
45 41 0.09 False
64 60 0.06 False
56 52 0.07 False
46 33 0.28 False
35 17 0.51 False
47 39 0.17 False
